**<hr><h>Capstone Project: Webscrapping and Exploring Downtown Toronto Borough in Canada<h><hr>**

**Setup Environment: Import modules**

In [177]:
! pip install beautifulsoup4
import pandas as pd
import requests
from bs4 import BeautifulSoup
import urllib.request
import numpy as np # library to handle data in a vectorized manner

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!conda update -n base -c defaults conda --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - conda


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    chardet-3.0.4              |        py37_1003         173 KB
    certifi-2019.9.11          |           py37_0         154 KB
    python-3.7.4               |       h265db76_1        36.5 MB
    pysocks-1.7.

**Scrapping dataset from web and setup pandas dataframe table**

In [4]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
res =  urllib.request.urlopen(url)
soup = BeautifulSoup(res,'html')
table = soup.find('table') 
table
A=[]
B=[]
C=[]
for row in table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))
df=pd.DataFrame(A,columns=['Postal Code'])
df['Borough']=B
df['Neighborhood']=C

**Data cleanup: Remove "Not assigned" Borough's and fill "Not assigned"Neighborhoods**

In [5]:
indexNames = df[df['Borough'] == 'Not assigned'].index #look through column to not assigned
df.drop(indexNames, inplace=True)#delete rows with not assisgned
df.Neighborhood.replace('Not assigned\n',df.Borough, inplace=True)

**Group PostalCode Neoghborhoods and compute new shape of dataframe**

In [6]:
df_pc = df.groupby(by=['Postal Code','Borough'],sort = False)['Neighborhood'].agg(', '.join).reset_index()

**Import Coordinates from CSV file**

In [7]:
df_gcoder = pd.read_csv('http://cocl.us/Geospatial_data', index_col=0)

**Combine Neighborhood and Coordinates dataframes via pandas inner join**

In [43]:
df_nbh=pd.merge(df_pc, df_gcoder, on='Postal Code', how='inner')

**Exploring Downtown Toronto,ON Neighborhood**

In [44]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_nbh['Borough'].unique()),
        df_nbh.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


**Geolocate Downtown Toronto**

In [45]:
address = 'Downtown Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.655115, -79.380219.


**Map Coordinates of Canada Boroughs**

In [196]:
# create map of Downtown Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, df_merged in zip(df_nbh['Latitude'], df_nbh['Longitude'], df_nbh['Borough'], df_nbh['Neighborhood']):
   #label = '{}, {}'.format(Neighborhood, borough)
   # label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        #popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

**Import Foursquare Credentials**

In [54]:
CLIENT_ID = '55ELXDS21BUOO4A2LBRM4T35UEOVCOFUYKG0KXTIPUI5OIWU' # your Foursquare ID
CLIENT_SECRET = 'KQO5CBRZRY1IDSUGQZOAPU0RR5QDG5A55DZAW4FOMSTPARLJ' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 55ELXDS21BUOO4A2LBRM4T35UEOVCOFUYKG0KXTIPUI5OIWU
CLIENT_SECRET:KQO5CBRZRY1IDSUGQZOAPU0RR5QDG5A55DZAW4FOMSTPARLJ


**Display Coordinates of 5 Downtown Toronto Boroughs***

In [50]:
df_dttor = df_nbh.loc[df_nbh['Borough'] == 'Downtown Toronto']
df_dttor.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
9,M5B,Downtown Toronto,"Ryerson\n, Garden District\n",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street\n,43.657952,-79.387383


**Display Coordinates of Toronto**

In [71]:
neighborhood_latitude = df_nbh.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_nbh.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_nbh.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


**Create URL for Neighbourhood within 500 miles Radius**

In [72]:
#Double-click __here__ for the solution.
#<!-- The correct answer is:
LIMIT = 100 # limit of number of venues returned by Foursquare API
#-->

#<!--
radius = 500 # define radius
#-->

#<!--
#\\ # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL
#--> 

'https://api.foursquare.com/v2/venues/explore?&client_id=55ELXDS21BUOO4A2LBRM4T35UEOVCOFUYKG0KXTIPUI5OIWU&client_secret=KQO5CBRZRY1IDSUGQZOAPU0RR5QDG5A55DZAW4FOMSTPARLJ&v=20180604&ll=43.7532586,-79.3296565&radius=500&limit=100'

**Define and display nearby Venues**

In [73]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [67]:
nbh_venues = getNearbyVenues(names=df_nbh['Neighborhood'],
                                   latitudes=df_nbh['Latitude'],
                                   longitudes=df_nbh['Longitude']
                                  )



Parkwoods
Victoria Village
Harbourfront, Regent Park
Lawrence Heights, Lawrence Manor
Queen's Park
Islington Avenue
Rouge, Malvern
Don Mills North

Woodbine Gardens, Parkview Hill
Ryerson
, Garden District

Glencairn

Cloverdale
, Islington, Martin Grove
, Princess Gardens, West Deane Park
Highland Creek, Rouge Hill, Port Union
Flemingdon Park, Don Mills South

Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens
, Eringate
, Markland Wood, Old Burnhamthorpe

Guildwood
, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks

Woburn
Leaside
Central Bay Street

Christie

Cedarbrae

Hillcrest Village
Bathurst Manor, Downsview North
, Wilson Heights
Thorncliffe Park
Adelaide
, King
, Richmond

Dovercourt Village, Dufferin

Scarborough Village
Fairview
, Henry Farm, Oriole

Northwood Park, York University
East Toronto
Harbourfront East
, Toronto Islands, Union Station
Little Portugal, Trinity
East Birchmount Park
, Ionview, Kennedy Park
Bayview Village
CFB Toron

**Print Shape and limited data**

In [74]:
print(nbh_venues.shape)
nbh_venues.head()

(2253, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Parkwoods,43.753259,-79.329656,TTC stop - 44 Valley Woods,43.755402,-79.333741,Bus Stop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


**Group Neighbourhood and show categories of venues**

In [75]:
nbh_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide\n, King\n, Richmond\n",100,100,100,100,100,100
Agincourt,3,3,3,3,3,3
"Agincourt North, L'Amoreaux East\n, Milliken, Steeles East\n",2,2,2,2,2,2
"Albion Gardens\n, Beaumond Heights, Humbergate\n, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",10,10,10,10,10,10
"Alderwood, Long Branch",8,8,8,8,8,8
"Bathurst Manor, Downsview North\n, Wilson Heights",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East\n",24,24,24,24,24,24
Berczy Park,57,57,57,57,57,57


In [79]:
print('There are {} uniques categories.'.format(len(nbh_venues['Venue Category'].unique())))

There are 270 uniques categories.


In [91]:
# one hot encoding
nbh_onehot = pd.get_dummies(nbh_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
nbh_onehot['Neighborhood'] = nbh_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [nbh_onehot.columns[-1]] + list(nbh_onehot.columns[:-1])
nbh_onehot = nbh_onehot[fixed_columns]

nbh_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [93]:
nbh_grouped = nbh_onehot.groupby('Neighborhood').mean().reset_index()
nbh_grouped.shape

(98, 270)

**Show Top 5 Venues by Neoghborhood**

In [98]:
num_top_venues = 5
for hood in nbh_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = nbh_grouped[nbh_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide
, King
, Richmond
----
              venue  freq
0       Coffee Shop  0.07
1              Café  0.05
2               Bar  0.04
3   Thai Restaurant  0.04
4  Asian Restaurant  0.03


----Agincourt----
                      venue  freq
0                    Lounge  0.33
1            Sandwich Place  0.33
2            Breakfast Spot  0.33
3  Mediterranean Restaurant  0.00
4        Miscellaneous Shop  0.00


----Agincourt North, L'Amoreaux East
, Milliken, Steeles East
----
                      venue  freq
0                Playground   0.5
1                      Park   0.5
2               Yoga Studio   0.0
3  Mediterranean Restaurant   0.0
4        Miscellaneous Shop   0.0


----Albion Gardens
, Beaumond Heights, Humbergate
, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                 venue  freq
0        Grocery Store   0.2
1          Pizza Place   0.1
2  Japanese Restaurant   0.1
3       Discount Store   0.1
4       Sandwich Place   0.1


----Alderwood

In [99]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

**Show top 10 Neighborhoods and their common venues**

In [170]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = nbh_grouped['Neighborhood']

for ind in np.arange(nbh_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(nbh_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide\n, King\n, Richmond\n",Coffee Shop,Café,Bar,Thai Restaurant,Restaurant,Asian Restaurant,Steakhouse,Hotel,Sushi Restaurant,American Restaurant
1,Agincourt,Lounge,Sandwich Place,Breakfast Spot,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Drugstore
2,"Agincourt North, L'Amoreaux East\n, Milliken, ...",Park,Playground,Women's Store,Doner Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
3,"Albion Gardens\n, Beaumond Heights, Humbergate...",Grocery Store,Discount Store,Pharmacy,Pizza Place,Fried Chicken Joint,Sandwich Place,Beer Store,Fast Food Restaurant,Japanese Restaurant,Women's Store
4,"Alderwood, Long Branch",Pizza Place,Gym,Skating Rink,Pharmacy,Pub,Sandwich Place,Coffee Shop,General Travel,General Entertainment,Donut Shop


**Run k-means to cluster the neighborhood into 7 clusters.**

In [249]:
# set number of clusters
kclusters = 7

nbh_grouped_clustering = nbh_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(nbh_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
#neighborhoods_venues_sorted.head()

array([1, 1, 2, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

**Add Clusters to Neighborhoods**

In [261]:
 #add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
#del df_nbh['Postal Code']
nbh_merger = df_nbh

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
nbh_merger = nbh_merger.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
indexNames = nbh_merger[nbh_merger['Cluster Labels'] == "NaN "].index #look through column to NaN
nbh_merger=nbh_merger.dropna()
nbh_merger['Cluster Labels'] = nbh_merger['Cluster Labels'].astype(int)
nbh_merger.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,Parkwoods,43.753259,-79.329656,2,Park,Food & Drink Shop,Bus Stop,Women's Store,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop
1,North York,Victoria Village,43.725882,-79.315572,1,Portuguese Restaurant,Coffee Shop,Pizza Place,French Restaurant,Hockey Arena,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
2,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,1,Coffee Shop,Park,Bakery,Café,Pub,Breakfast Spot,Mexican Restaurant,Theater,Restaurant,French Restaurant
3,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,1,Furniture / Home Store,Clothing Store,Women's Store,Coffee Shop,Boutique,Miscellaneous Shop,Event Space,Accessories Store,Vietnamese Restaurant,Comic Shop
4,Queen's Park,Queen's Park,43.662301,-79.389494,1,Coffee Shop,Park,Gym,Burger Joint,Diner,Smoothie Shop,Seafood Restaurant,Sandwich Place,Burrito Place,Café


<hr>**Create Map of the Neighbour Clustering**

In [255]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(nbh_merger['Latitude'], nbh_merger['Longitude'], nbh_merger['Neighborhood'], nbh_merger['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters